<a href="https://colab.research.google.com/github/trkfz2/M2020/blob/master/NLP_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade nltk

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=129e8c2624fd051f4f8983f70bb8de8f95012935289d831f1a04abbbfdb19b63
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
import nltk
nltk.__version__

'3.4.5'

In [0]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt


      Unzipping tokenizers/punkt.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

# 1. Split the dataset into a training and a testing subset

Use the category “title” as the testing set and the categories “comment” and “post” as the training set. The short length of titles will make them good candidates later on as seeds for text generation.



In [0]:
import pandas as pd

### Load the data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv('drive/My Drive/Manning Project/NLP/stackexchange_812k.tokenized.csv').sample(frac = 1, random_state = 0).reset_index(drop = True)

In [0]:
assert data.shape == (789649, 7), "The dataset does not have the right dimensions"

In [0]:
data.columns

Index(['post_id', 'parent_id', 'comment_id', 'text', 'category', 'tokens',
       'n_tokens'],
      dtype='object')

In [0]:
pd.options.display.max_colwidth = 500

In [0]:
data.head()

,post_id,parent_id,comment_id,text,category,tokens,n_tokens
0,298828,NaN,567970.0,"That's not an option atm. This data definitely provides some evidence for length of adherence, I just don't know how to use it.",comment,"that ' s not an option atm . this data definitely provides some evidence for length of adherence , i just don ' t know how to use it .",30
1,142749,142741.0,NaN,"already provided a link to the discussion containing the theoretical aspects. Here is a quick pratical example of how one would do it in R. Please also have a look at these documents which contain the theory as well as examples Simultaneous Inference in General Parametric Models and Additional multcomp Examples . We will use the mtcars dataset and build a linear regression model containing three variables cyl Number of cylinders , disp Displacement and hp Horsepower to predict the variable m...",post,"already provided a link to the discussion containing the theoretical aspects . here is a quick pratical example of how one would do it in r . please also have a look at these documents which contain the theory as well as examples simultaneous inference in general parametric models and additional multcomp examples . we will use the mtcars dataset and build a linear regression model containing three variables cyl number of cylinders , disp displacement and hp horsepower to predict the variable...",332
2,49674,NaN,96088.0,As for I would say this question is off-topic and unlikely to help future visitors interested in statistical science in its theoretical and applied aspects.,comment,as for i would say this question is off - topic and unlikely to help future visitors interested in statistical science in its theoretical and applied aspects .,28
3,264715,NaN,NaN,"I attended a conference on ML and Data Science and I have a general question that was not answered in the conference. If we have a continuous variable, let's say age. What is the best way to handle this variable. These are my thoughts, please let me know if they nonsense, but in general I think it is a very important and useful topic that has not been discussed in the detail that I need it How should you decide on the number of bins? Would it be best to choose an arbitrary number of bins and...",post,"i attended a conference on ml and data science and i have a general question that was not answered in the conference . if we have a continuous variable , let ' s say age . what is the best way to handle this variable . these are my thoughts , please let me know if they nonsense , but in general i think it is a very important and useful topic that has not been discussed in the detail that i need it how should you decide on the number of bins ? would it be best to choose an arbitrary number of...",258
4,45344,NaN,88126.0,"Oh well, I actually meant equivalence in terms of only observed variables -- essentially maxing or more commonly, marginalizing out latent variables and then establishing equivalence.",comment,"oh well , i actually meant equivalence in terms of only observed variables -- essentially maxing or more commonly , marginalizing out latent variables and then establishing equivalence .",29


In [0]:
list(data['category'].unique())

['comment', 'post', 'title']

Training set

In [0]:
training = data[data.category.isin(["post","comment"]) ]

In [0]:
training.shape

(705964, 7)

In [0]:
training.category.value_counts()

comment    540587
post       165377
Name: category, dtype: int64

Testing set

In [0]:
testing = data[data.category == 'title']

In [0]:
testing.shape

(83685, 7)

# 2. Build the matrix of prefix - word frequencies

Use the ngrams function from nltk.utils to generate all n-grams from the corpus
Set the following left_pad_symbol = `<s>` and right_pad_symbol = `</s>`


In [0]:
# we need only tokens for training

training_text = training["tokens"]

In [0]:
training_text.shape

(705964,)

In [0]:
# shown an example
training_text[0]

"that ' s not an option atm . this data definitely provides some evidence for length of adherence , i just don ' t know how to use it ."

In [0]:
from nltk import tokenize
from nltk.tokenize import sent_tokenize

In [0]:
from nltk.util import pad_sequence

In [0]:
def prepare_sent(text):

  # split text into sentences
  sent = tokenize.sent_tokenize(text)

  # split into tokens, pad sentences for bigrams
  pad_sent = [list(pad_sequence(item.split(), n = 2, pad_left = True, pad_right = True, left_pad_symbol="<s>", right_pad_symbol="</s>")) for item in sent]
  
  return pad_sent


In [0]:
# split comments and posts into sentences, pad both ends

pad_tr_text = [prepare_sent(item) for item in training_text]

In [0]:
# show one post/comment

pad_tr_text[0]

[['<s>', 'that', "'", 's', 'not', 'an', 'option', 'atm', '.', '</s>'],
 ['<s>',
  'this',
  'data',
  'definitely',
  'provides',
  'some',
  'evidence',
  'for',
  'length',
  'of',
  'adherence',
  ',',
  'i',
  'just',
  'don',
  "'",
  't',
  'know',
  'how',
  'to',
  'use',
  'it',
  '.',
  '</s>']]

In [0]:
from nltk.lm.preprocessing import flatten

In [0]:
# flatten rows

flat_pad_rows = list(flatten(pad_tr_text))

In [0]:
# all sentences from the first post/comment

flat_pad_rows[0:2]

[['<s>', 'that', "'", 's', 'not', 'an', 'option', 'atm', '.', '</s>'],
 ['<s>',
  'this',
  'data',
  'definitely',
  'provides',
  'some',
  'evidence',
  'for',
  'length',
  'of',
  'adherence',
  ',',
  'i',
  'just',
  'don',
  "'",
  't',
  'know',
  'how',
  'to',
  'use',
  'it',
  '.',
  '</s>']]

In [0]:
# flatten all sentences into a list of words

flat_pad_sent = list(flatten(flat_pad_rows))

In [0]:
len(flat_pad_sent)

54058369

In [0]:
flat_pad_sent[0:20]

['<s>',
 'that',
 "'",
 's',
 'not',
 'an',
 'option',
 'atm',
 '.',
 '</s>',
 '<s>',
 'this',
 'data',
 'definitely',
 'provides',
 'some',
 'evidence',
 'for',
 'length',
 'of']

----

Find all bigrams

In [0]:
from nltk.util import bigrams

In [0]:
padded_bigrams = list(bigrams(flat_pad_sent))

In [0]:
len(padded_bigrams)

54058368

In [0]:
# show one bigram

padded_bigrams[1]

('that', "'")

-----


N-gram models

In [0]:
from collections import Counter, defaultdict

In [0]:
# unigram model

model_uni = defaultdict(int)

In [0]:
# unigram counts

for unigram in flat_pad_sent:
  model_uni[unigram] +=1

In [0]:
len(model_uni)

141969

In [0]:
model_uni['that']

584957

In [0]:
model_uni['because']

55682

In [0]:
# bigram model

model_bigram = defaultdict(lambda: 0)

In [0]:
# bigram counts
for bigram in padded_bigrams:
  model_bigram[(bigram[0], bigram[1])] +=1

In [0]:
len(model_bigram)

3314199

Log probabilities

In [0]:
import math

In [0]:
# bigram log probs

model_bigram_prob = defaultdict()

for bigram, count in model_bigram.items():
  if type(bigram) == int:
    continue
  
  uni_count = model_uni[bigram[0]]
  if uni_count == 0:
    continue
  
  prob = count/uni_count
  log_prob = math.log(prob)
  
  model_bigram_prob[bigram] = log_prob

In [0]:
# bigram log probs (laplace smoothing)

model_bigram_prob_smooth = defaultdict()

for bigram, count in model_bigram.items():

  if type(bigram) == int:
    continue
  
  uni_count = model_uni[bigram[0]]
  if uni_count == 0:
    continue
  
  # laplace smoothing
  prob = (count + 1)/(uni_count + len(model_uni))
  log_prob = math.log(prob)

  model_bigram_prob_smooth[bigram] = log_prob

Write a text generation function:
- takes a bigram as input and generates the next token
- iteratively slide the prefix over the generated text so that the new prefix includes the most recent token; 
- generates the next token
- to generate each next token, sample the list of words associated with the prefix using the probability distribution of the prefix.
- stop the text generation when a certain number of words have been generated or the latest token is a </s>.



In [0]:
# only unigram input

def predict_word(model, start = '<s>', tau = 0.5):
   
   # find all bigrams that begin with 'start'
   # return max prob word

   probs = list(model.values())
   bigrams = list(model.keys())
   next_bigrams = [item for item in bigrams if item[0] == start]
   
   #temperarture sampling
   prob_sum = [math.pow(math.exp(item), 1/tau) for item in probs]
   prob_sum = sum(prob_sum)

   max_prob = 0
   next_word = ''
  
   for bigram in next_bigrams:
       prob = math.exp(model[bigram])
       
       # temperature sampling, tau
       prob = math.pow(prob, 1/tau) / prob_sum
       
       if prob >= max_prob:
           max_prob = prob
           next_word = bigram[1]

   return next_word

In [0]:
# only unigram input

def generate_text(model, max_length = 7, start = '<s>', end = '</s>', tau = 0.5):
  
  word = ''
  num_words = 0
  generated_text = start
  
  while word != end:
    word = predict_word(model = model, start = start, tau = tau)
    generated_text = generated_text + ' ' + word
    #print(word)
    start = word
    num_words +=1
    if num_words > max_length:
      break

  return generated_text

In [0]:
s = generate_text(model_bigram_prob, tau = 0.5)
s

"<s> i ' s a good idea of the"

In [0]:
s = generate_text(model_bigram_prob, start = 'probability', tau = 0.5)
s

'probability of the same as a good idea of'

Write a function that can estimate the probability of a sentence and use it to select the most probable sentence out of several candidate sentences

Split the sentence into trigrams and use the chain rule to calculate the probability of the sentence as a product of the bigrams - tokens probabilities



In [0]:
def bigram_sent_split(sentence): 
  sent = prepare_sent(sentence)
  sent_bigrams = list(bigrams(sent[0]))

  return sent_bigrams
  

In [0]:
import math
def sent_prob(sentence, model):
  sent = bigram_sent_split(sentence)
  #sent.reverse()

  sent_prob = 0
  for bigram in sent:
    prob = model[bigram]

    #print(prob)
    sent_prob  = sent_prob + prob

  return math.exp(sent_prob)

In [0]:
sprob = sent_prob("thanks for help", model_bigram_prob) # TO DO: unknown bigrams!

print(sprob)
print('{:.20f}'.format(sprob))


8.344074841251085e-08
0.00000008344074841251


In [0]:
sprob = sent_prob("thanks for help", model_bigram_prob_smooth) # TO DO: unknown bigrams!

print(sprob)
print('{:.20f}'.format(sprob))

3.622204560930194e-09
0.00000000362220456093


Implement the perplexity scoring function for a given sentence and for the training corpus.
Implement Additive Laplace smoothing to give a non zero probability to missing prefix - token combinations when calculating perplexity.

Calculate the perplexity of the language model on the test set composed of titles.



In [0]:
def sent_perplexity(sentence, model):

  sent = prepare_sent(sentence)
  #print(sent)
  s_length = len(sent[0])
  
  #print("length:", s_length)
  
  sent_bigrams = list(bigrams(sent[0]))
  #print(sent_bigrams)
  
  sent_prob = 0
  for bigram in sent_bigrams:
    log_prob = model[bigram]
    #print(prob)
    sent_prob  = sent_prob + log_prob

  
  sent_prob = math.exp(sent_prob)
  #print("sent_prob:", sent_prob)

  perplexity = math.pow(sent_prob, -1/(s_length-1))

  return perplexity
  




In [0]:
# selected sentence
s

'probability of the same as a good idea of'

In [0]:
s_perplex = sent_perplexity(s, model_bigram_prob)
s_perplex

34.69041524825544

In [0]:
# with laplace smoothing

s_perplex = sent_perplexity(s, model_bigram_prob_smooth)
s_perplex

70.17752957486314

-----

Try to improve the perplexity score of your model by
modifying the preprocessing phase of the corpus
increasing or decreasing number of tokens in the model (bi grams, 4-grams, …)
varying the delta parameter in the Additive Laplace smoothing step.